**PREDICCIÓN DE TIPO DE PERSONALIDAD DE UNA PERSONA**

El concepto de las "16 personalidades" proviene del indicador de tipo Myers-Briggs (MBTI), que es una teoría fue desarrollada por Isabel Briggs Myers y su madre, Katharine Cook Briggs, basada en las teorías del psiquiatra Carl Jung. Este modelo clasifica la personalidad en 16 tipos distintos, que resultan de combinar cuatro pares de preferencias:

Extraversión (E) - Introversión (I): Indica dónde las personas obtienen su energía: de la interacción con el mundo externo (E) o del mundo interno de ideas y reflexiones (I).

Sensación (S) - Intuición (N): Describe cómo las personas procesan información. Los tipos "S" se centran en detalles concretos y realidades prácticas, mientras que los tipos "N" se enfocan en patrones y posibilidades futuras.

Pensamiento (T) - Sentimiento (F): Define cómo las personas toman decisiones. Los "T" prefieren decisiones lógicas y objetivas, mientras que los "F" priorizan valores y consideraciones personales.

Juicio (J) - Percepción (P): Indica cómo las personas manejan el mundo externo: con estructura y planificación (J) o de manera abierta y flexible (P).

Cada tipo de personalidad es representado por una combinación de estas cuatro letras (por ejemplo, INTJ, ENFP, ESTJ), dando un total de 16 combinaciones posibles. (Luego se comprueba en el dataset).

El MBTI se usa en diferentes ámbitos, como en desarrollo personal, orientación vocacional y en el ámbito laboral para mejorar la dinámica de equipo.


**INFORMACIÓN IMPORTANTE**

En este archivo divido el código en celdas para posteriormente explicar cada paso que hago, es decir, la explicación de cada celda va DESPUÉS de la misma.

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import ggplot, aes, geom_bar, labs, theme_void
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import pickle
import gradio as gr
from imblearn.over_sampling import SMOTE

/Users/albertoyesaresmartinez/VSCode Projects/astral/proyecto_final_alberto_yesares/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [3]:
df = pd.read_csv('data.csv')

In [4]:
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

In [5]:
df = pd.get_dummies(df, columns=['Interest'])

In [6]:
scaler = StandardScaler()
num_cols = ['Age', 'Introversion Score', 'Sensing Score', 'Thinking Score', 'Judging Score']
df[num_cols] = scaler.fit_transform(df[num_cols])

In [7]:
# Cargar datos
X = df.drop('Personality', axis=1)
y = df['Personality']

# Codificar la variable objetivo
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# División estratificada en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, stratify=y, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Verificar la distribución de clases después de SMOTE
print(pd.Series(y_train_resampled).value_counts())

9     27523
2     27523
3     27523
1     27523
0     27523
11    27523
5     27523
8     27523
13    27523
15    27523
4     27523
10    27523
7     27523
12    27523
14    27523
6     27523
Name: count, dtype: int64


In [8]:
# Entrenar el modelo KNN con los datos resampleados
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_resampled, y_train_resampled)

KNeighborsClassifier()

Entrenamiento del modelo. Utilizo RandomForest. He probado en poner varios n_estimators y a partir de 500 no mejora la precisión del modelo.

In [10]:
with open('random_forest_model_v5.pkl', 'wb') as f:
    pickle.dump(knn, f)

Guardo el modelo en el fichero random_forest_modelo_v2. 

In [11]:
y_pred = knn.predict(X_test)
print("Precisión:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Precisión: 0.7723031273181588
              precision    recall  f1-score   support

           0       0.63      0.81      0.71       777
           1       0.89      0.78      0.83      6881
           2       0.63      0.78      0.70       557
           3       0.86      0.77      0.81      4944
           4       0.46      0.65      0.54       111
           5       0.55      0.75      0.63       966
           6       0.49      0.65      0.56        78
           7       0.52      0.73      0.61       667
           8       0.63      0.78      0.70       584
           9       0.84      0.78      0.81      4942
          10       0.62      0.74      0.68       384
          11       0.79      0.79      0.79      3427
          12       0.49      0.58      0.53        74
          13       0.52      0.75      0.62       691
          14       0.44      0.50      0.47        52
          15       0.52      0.71      0.60       478

    accuracy                           0.77     25

In [12]:
# Cargar el modelo desde el archivo
with open('random_forest_model_v5.pkl', 'rb') as f:
    model = pickle.load(f)

# Asumiendo que ya tienes el LabelEncoder entrenado
label_encoder = LabelEncoder()
label_encoder.fit(['ENFJ', 'ENFP', 'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP', 
                   'INFJ', 'INFP', 'INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP'])

# Crear un StandardScaler para escalar las características numéricas en la predicción
scaler = StandardScaler()

# Diccionario para traducir intereses del español al inglés
interest_translation = {
    "Desconocido": "Unknown",
    "Artes": "Arts",
    "Otros": "Others",
    "Tecnología": "Technology",
    "Deportes": "Sports"
}

# Extrae los nombres exactos de las características usadas durante el entrenamiento
feature_names = model.feature_names_in_

# Función de predicción que usa el modelo y genera el link de información
def predict_personality(age, gender, education, introversion_score, sensing_score, thinking_score, judging_score, interest):
    # Procesamiento de entrada
    gender = 0 if gender == 'Masculino' else 1  # Mapear género a 0/1
    education = 1 if education == 'Soy licenciado' else 0  # Mapear educación a 1/0

    # Traducir interés al inglés para el modelo
    interest = interest_translation[interest]
    
    # Codificar el interés como variable dummy
    interests = ['Unknown', 'Arts', 'Others', 'Technology', 'Sports']
    interest_encoded = [1 if interest == i else 0 for i in interests]

    # Escalar las variables numéricas
    input_data = np.array([[age, introversion_score, sensing_score, thinking_score, judging_score]])
    input_data_scaled = scaler.fit_transform(input_data)

    # Concatenar variables escaladas con las variables categóricas
    full_input = np.concatenate((input_data_scaled.flatten(), [gender, education] + interest_encoded))
    
    # Convertir full_input a un DataFrame con los nombres de las características originales
    full_input_df = pd.DataFrame([full_input], columns=feature_names)
    
    # Realizar la predicción
    prediction_encoded = model.predict(full_input_df)[0]
    
    # Decodificar la predicción de vuelta a la etiqueta original de personalidad
    personality_type = label_encoder.inverse_transform([prediction_encoded])[0]
    
    # Generar el enlace de información
    link = f"https://www.16personalities.com/es/personalidad-{personality_type.lower()}"
    
    # Mensaje final con la predicción y el enlace
    result_message = f"Tu tipo de personalidad es: {personality_type}\n\n[Más información sobre tu tipo de personalidad]({link})"
    
    return result_message


# Descripciones para los sliders con etiquetas de extremos
info_introversion = "Introversión (0)  vs. Extroversión (10)"
info_sensing = "Sensación (0)  vs. Intuición (10)"
info_thinking = "Pensamiento (0)  vs. Sentimiento (10)"
info_judging = "Percepción (0)  vs. Juicio (10)"

# Configuración de la interfaz de Gradio
iface = gr.Interface(
    fn=predict_personality,
    inputs=[
        gr.Slider(18, 57, step=1, label="Edad"),
        gr.Radio(choices=["Masculino", "Femenino"], label="Género"),
        gr.Radio(choices=["Soy licenciado", "No lo soy"], label="Educación"),
        gr.Slider(0, 10, step=1, label=info_introversion, info="Indica tu nivel de preferencia por la reflexión y la introspección"),
        gr.Slider(0, 10, step=1, label=info_sensing, info="Mide tu inclinación hacia la observación de hechos y detalles concretos"),
        gr.Slider(0, 10, step=1, label=info_thinking, info="Evalúa tu preferencia por tomar decisiones de manera lógica y objetiva"),
        gr.Slider(0, 10, step=1, label=info_judging, info="Mide tu tendencia a estructurar y planificar tu vida"),
        gr.Dropdown(choices=["Desconocido", "Artes", "Otros", "Tecnología", "Deportes"], label="Intereses")
    ],
    outputs="markdown",  # Configuramos como markdown para soportar enlaces
    title="Predicción de Tipo de Personalidad",
    description="Introduce los valores correctos para ver cuál es tu personalidad."
)

# Ejecutar la interfaz
iface.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
